In [6]:
import requests
from transformers import pipeline

def get_named_entities(text):
    # Load the NER pipeline
    ner_pipeline = pipeline("ner", aggregation_strategy="simple")

    # Perform NER on the string
    entities = ner_pipeline(text)
    
    # Extract and print the recognized entities
    named_entities = []
    for entity in entities:
        named_entities.append(entity['word'])
    return named_entities

def get_dbpedia_entry(entity):
    query = f"""
    SELECT ?abstract WHERE {{
        ?entity rdfs:label "{entity}"@en .
        ?entity dbo:abstract ?abstract .
        FILTER (lang(?abstract) = 'en')
    }} LIMIT 1
    """
    url = "https://dbpedia.org/sparql"
    headers = {
        "Accept": "application/sparql-results+json"
    }
    response = requests.get(url, headers=headers, params={"query": query})
    data = response.json()
    if data["results"]["bindings"]:
        return data["results"]["bindings"][0]["abstract"]["value"]
    else:
        return None

# LLM Agent
def llm_agent(prompt, ctxt):
    qa_pipeline = pipeline("question-answering")
    return qa_pipeline(question=prompt, context=ctxt)['answer']

# Example usage
question = "Who is Barack Obama?"
named_entities = get_named_entities(question)
print(named_entities)

entity = named_entities[0]
entry = get_dbpedia_entry(entity)

print("prompt",question)
print("context",entry)
print(llm_agent(question,entry))

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['Barack Obama']


No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


prompt Who is Barack Obama?
context Barack Hussein Obama II (/bəˈrɑːk huːˈseɪn oʊˈbɑːmə/ bə-RAHK hoo-SAYN oh-BAH-mə; born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, Obama was the first African-American president of the United States. He previously served as a U.S. senator from Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004, and previously worked as a civil rights lawyer before entering politics. Obama was born in Honolulu, Hawaii. After graduating from Columbia University in 1983, he worked as a community organizer in Chicago. In 1988, he enrolled in Harvard Law School, where he was the first black president of the Harvard Law Review. After graduating, he became a civil rights attorney and an academic, teaching constitutional law at the University of Chicago Law School from 1992 to 2004. Turning to elective politics, he represented the 13th district i